<h1>Exercise 1&2: Building a MINST Classifier with 97%+ accuracy and shift images around </h1>

In [1]:
#importing stuff
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_openml

<h2> Getting the data </h2>

In [2]:
mnist = fetch_openml('mnist_784', version=1)
X, y = mnist["data"], mnist["target"]

In [3]:
y = y.astype(np.uint8)

In [4]:
 X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

In [5]:
#using code I got from stackoverflow
#https://stackoverflow.com/questions/30399534/shift-elements-in-a-numpy-array
def shift(digit, n, vertical=False):
    digit = digit.copy().values.reshape(28,28)
    e = np.empty_like(digit)
    if vertical:
        if n >= 0:
            e[:n, :] = 0.0
            e[n:, :] = digit[:-n]
        else:
            e[n:, :] = 0.0
            e[:n, :] = digit[-n:]
    else:
        if n >= 0:
            e[:n] = 0.0
            e[n:] = digit[:-n]
        else:
            e[n:] = 0.0
            e[:n] = digit[-n:]
    return pd.Series(e.flatten())

In [6]:
def print_digit(digit):
    digit = digit.values.reshape(28,28)
    plt.imshow(digit, cmap="binary")
    plt.axis("off")
    plt.show()

<h2> Create the dataset with the shifts </h2>

In [7]:
def create_label_set(labels):
    new_dims = 5*labels.shape[0]
    new_dataset = pd.Series(np.zeros(new_dims))
    for i in range(labels.shape[0]):
        for j in range(0,5):
            new_dataset.iloc[5*i+j] = labels.iloc[i]
    
    return new_dataset

In [8]:
def create_shifted_dataset(dataset):
    new_dims = (5*dataset.shape[0], dataset.shape[1])
    new_dataset = pd.DataFrame(np.zeros(new_dims))
    for i in range(dataset.shape[0]):
        new_dataset.iloc[5*i] = dataset.iloc[i]
        new_dataset.iloc[5*i+1] = shift(dataset.iloc[i],1,True).values
        new_dataset.iloc[5*i+2] = shift(dataset.iloc[i],-1,True).values
        new_dataset.iloc[5*i+3] = shift(dataset.iloc[i],1,False).values
        new_dataset.iloc[5*i+4] = shift(dataset.iloc[i],-1,False).values
    
    return new_dataset

In [9]:
shifted_dataset = create_shifted_dataset(X_train)

In [10]:
extra_labels = create_label_set(y_train)

In [11]:
print(type(shifted_dataset))

<class 'pandas.core.frame.DataFrame'>


<h1> Training the Model </h1>

In [12]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'n_neighbors': [5,10],'weights':['uniform', 'distance']}
]

knn_clf = KNeighborsClassifier(n_neighbors=6, weights='distance')

knn_clf.fit(shifted_dataset, extra_labels)

KNeighborsClassifier(n_neighbors=6, weights='distance')

<h1> Testing the Model </h1>

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(knn_clf, shifted_dataset, extra_labels, cv=3, scoring="accuracy")